Постройте программу, которая в качестве входа принимает произвольную КС-грамматику $G = (N, \sum, P, S)$ без
$\epsilon$ -правил и преобразует ее в эквивалентную КС-грамматику $G' = (N, \sum, P', S)$ без $\epsilon$ -правил и без цепных правил.


In [6]:
def debug_print(func):
    def wrapper(*args, **kwargs):
        print(' BEGIN DEBUG PRINT '.center(64, '='))
        res = func(*args, **kwargs)
        print(' END DEBUG PRINT '.center(64, '='))
        return res
    return wrapper

In [1]:
from functools import reduce

class GrammarRule:
    def __init__(self, l: str, r:list[str]) -> None:
        self.lhs = l
        self.rhs = r.copy()
    
    def __repr__(self) -> str:
        return f'{self.lhs} -> {self.rhs}'
    
    def __str__(self) -> str:
        return f'{self.lhs} -> {self.rhs}'

In [2]:
class Grammar:
    nonterminal = []
    terminal = []
    rules = []
    axiom = ''

    def __init__(self, nonterm: list[str], term: list[str], rules: list[GrammarRule], axiom: str) -> None:
        self.nonterminal = nonterm
        self.terminal = term
        self.rules = rules
        self.axiom = axiom
    
    def print(self, std=True, md=True):
        output = ""
        output += "[{:}] [{:}] {:}\n".format(",".join(sorted(self.nonterminal)), ",".join(sorted(self.terminal)), self.axiom) 

        for nt in sorted(self.nonterminal):
            rules = list(filter(lambda x: x.lhs == nt, self.rules))
            if not len(rules):
                continue
            
            output += '\n' + ( nt + " -> " + " | ".join(sorted(map(lambda x: " ".join(x.rhs) if len(x.lhs) else r'\epsilon', rules))))
        if std:
            print(output)
        
        if md:
            output = "\n\n".join(map(lambda x: '$' + x.replace('->', '\\rightarrow') + '$' if len(x) else '', output.split('\n')))

        return output
    
    def show(self):
        import IPython
        return IPython.display.Markdown(self.print(std=False))
    
    @staticmethod
    def from_text(text):
        nt, t, a = text.split('\n')[0].split(' ')
        nt = nt[1:-1].split(',')
        t = t[1:-1].split(',')
        
        rules = []

        for string in text.split('\n')[1:]:
            if not len(string):
                continue
            
            left, right = string.split(' -> ')

            for rule in right.split(' | '):
                data = rule.split(' ')
                if data[0] == r'\epsilon':
                    data = []
                rules.append(
                    GrammarRule(left, data)
                )
        return Grammar(nt, t, rules, a)

In [3]:
# grammar = Grammar(
#     nonterm=['E', 'T', 'F', 'G', 'R', 'U'],
#     term=['+', '-', '(', ')', 'a'],
#     rules=[
#         GrammarRule('E', ['E', '+', 'T']),
#         GrammarRule('E', ['T']),
#         GrammarRule('T', ['T', '*', 'F']),
#         GrammarRule('T', ['F']),
#         GrammarRule('F', ['a']),
#         GrammarRule('F', ['G']),
#         GrammarRule('G', ['F']),
#         GrammarRule('G', ['G', '+', 'R']),
#         GrammarRule('G', []),
#         GrammarRule('F', ['(', 'E', ')']),
#         GrammarRule('U', ['F']),
#     ],
#     axiom='E'
# )

grammar = Grammar(
    nonterm=['E', 'T', 'F'],
    term=['+', '-', '(', ')', 'a'],
    rules=[
        GrammarRule('E', ['E', '+', 'T']),
        GrammarRule('E', ['T']),
        GrammarRule('T', ['T', '*', 'F']),
        GrammarRule('T', ['F']),
        GrammarRule('F', ['a']),
        GrammarRule('F', ['(', 'E', ')']),
    ],
    axiom='E'
)

In [4]:
grammar.show()

$[E,F,T] [(,),+,-,a] E$



$E \rightarrow E + T | T$

$F \rightarrow ( E ) | a$

$T \rightarrow F | T * F$

In [7]:
@debug_print
def remove_left_recursion(g: Grammar):
    new_rules: list[GrammarRule] = g.rules.copy()

    nonterminals = g.nonterminal.copy()
    # Расположили нетерминалы в некотором порядке
    print('1. Nonterminals: ', g.nonterminal)
    nnt = ['E', 'T', 'F']
    # for i, i_nt in enumerate(g.nonterminal):
    for i, i_nt in enumerate(nnt):
        # for j_nt in g.nonterminal[:i]:
        for j_nt in nnt[:i]:
            # Отобрать продукции вида A_i --> A_j
            rules = [
                rule for rule in new_rules if rule.lhs == i_nt and len(rule.rhs) and rule.rhs[0] == j_nt
            ]
            print(f'2. Ищем продукции: {i_nt} ---> {j_nt}')
            if rules: print(f'3. Замена продукции: {rules[0]}')
            for rule in rules:
                new_rules.remove(rule)
                m_rules = [rule for rule in new_rules if rule.lhs == j_nt]
                for m_rule in m_rules:
                    new_rule = GrammarRule(i_nt, m_rule.rhs + rule.rhs[1:])
                    print(f'4. Заменяем правилом: {new_rule}')
                    new_rules.append(new_rule)
        print(f'5. Переходим к устранению непосредственной левой рекурсии по {i_nt}')
        # Отбираем правила для выбранного нетерминала
        rules = [rule for rule in new_rules if rule.lhs == i_nt] 
        need_modify = False
        for rule in rules:
            # Если правило непустое и леворекурсивно
            if len(rule.rhs) and rule.rhs[0] == i_nt:
                need_modify = True
                print(f'Правило {rule} леворекурсивно')
                break

        if need_modify:
            # Вводим новый нетерминал
            new_nt = i_nt + "'"
            nonterminals += [new_nt]

            for rule in rules:
                new_rules.remove(rule)
                print(f'Заменяем правило {rule}')
                if len(rule.rhs) and rule.rhs[0] == i_nt:
                    new_rule_1 = GrammarRule(new_nt, rule.rhs[1:]) 
                    new_rule_2 = GrammarRule(new_nt, rule.rhs[1:] + [new_nt])
                    new_rules += [new_rule_1, new_rule_2]
                    print(f'Вводим новые правила:')
                    print(new_rule_1)
                    print(new_rule_2)
                    print()
                else:
                    new_rule_1 = GrammarRule(rule.lhs, rule.rhs.copy()) 
                    new_rule_2 = GrammarRule(rule.lhs, rule.rhs + [new_nt])
                    new_rules += [new_rule_1, new_rule_2]
                    print(f'Вводим новые правила:')
                    print(new_rule_1)
                    print(new_rule_2)
                    print()
    return Grammar(nonterminals, g.terminal.copy(), new_rules, g.axiom)

display(grammar.show())
nlr = remove_left_recursion(grammar)
nlr.show()

$[E,F,T] [(,),+,-,a] E$



$E \rightarrow E + T | T$

$F \rightarrow ( E ) | a$

$T \rightarrow F | T * F$

====================== BEGIN DEBUG PRINT =======================
1. Nonterminals:  ['E', 'T', 'F']
5. Переходим к устранению непосредственной левой рекурсии по E
Правило E -> ['E', '+', 'T'] леворекурсивно
Заменяем правило E -> ['E', '+', 'T']
Вводим новые правила:
E' -> ['+', 'T']
E' -> ['+', 'T', "E'"]

Заменяем правило E -> ['T']
Вводим новые правила:
E -> ['T']
E -> ['T', "E'"]

2. Ищем продукции: T ---> E
5. Переходим к устранению непосредственной левой рекурсии по T
Правило T -> ['T', '*', 'F'] леворекурсивно
Заменяем правило T -> ['T', '*', 'F']
Вводим новые правила:
T' -> ['*', 'F']
T' -> ['*', 'F', "T'"]

Заменяем правило T -> ['F']
Вводим новые правила:
T -> ['F']
T -> ['F', "T'"]

2. Ищем продукции: F ---> E
2. Ищем продукции: F ---> T
5. Переходим к устранению непосредственной левой рекурсии по F
======================= END DEBUG PRINT ========================


$[E,E',F,T,T'] [(,),+,-,a] E$



$E \rightarrow T | T E'$

$E' \rightarrow + T | + T E'$

$F \rightarrow ( E ) | a$

$T \rightarrow F | F T'$

$T' \rightarrow * F | * F T'$

In [99]:
display(grammar.show())
display(nlr.show())

$[E,F,T] [(,),+,-,a] E$



$E \rightarrow E + T | T$

$F \rightarrow ( E ) | a$

$T \rightarrow F | T * F$

$[E,E',F,T,T'] [(,),+,-,a] E$



$E \rightarrow T | T E'$

$E' \rightarrow + T | + T E'$

$F \rightarrow ( E ) | a$

$T \rightarrow F | F T'$

$T' \rightarrow * F | * F T'$

In [ ]:
grammar = Grammar(
    nonterm=['E', 'T', 'F'],
    term=['+', '-', '(', ')', 'a'],
    rules=[
        GrammarRule('E', ['E', '+', 'T']),
        GrammarRule('E', ['T']),
        GrammarRule('T', ['T', '*', 'F']),
        GrammarRule('T', ['F']),
        GrammarRule('F', ['a']),
        GrammarRule('F', ['(', 'E', ')']),
    ],
    axiom='E'
)

In [106]:
def remove_chain_rules(grammar: Grammar) -> Grammar:
    # Шаг 1: Построение N_A для каждого нетерминала A
    N = {nt: set([nt]) for nt in grammar.nonterminal}  # Начальное множество N_A={A}
    changes = True
    while changes:  # Повторяем до тех пор, пока есть изменения
        changes = False
        for A in grammar.nonterminal:
            current_N = N[A].copy()  # Tекущее множество N_i-1
            for B in current_N:
                for rule in grammar.rules:
                    # Если правило имеет вид B -> C и C - нетерминал, то добавляем C в N_A
                    if rule.lhs == B and len(rule.rhs) == 1 and rule.rhs[0] in grammar.nonterminal:
                        if rule.rhs[0] not in N[A]:
                            N[A].add(rule.rhs[0])
                            changes = True

    # Шаг 2: Построение нового набора правил P'
    new_rules = []
    for A in grammar.nonterminal:
        for rule in grammar.rules:
            # Добавляем правило A -> α, если оно не является цепным правилом
            if len(rule.rhs) != 1 or rule.rhs[0] not in grammar.nonterminal:
                # Если B входит в N_A, добавляем A -> α для каждого такого B
                if rule.lhs in N[A]:
                    new_rules.append(GrammarRule(A, rule.rhs.copy()))

    # Шаг 3: Создаем новую грамматику без цепных правил
    return Grammar(grammar.nonterminal, grammar.terminal, new_rules, grammar.axiom)

# Теперь можно использовать функцию для создания новой грамматики без цепных правил
new_grammar = remove_chain_rules(grammar)
new_grammar.show()

$[E,F,T] [(,),+,-,a] E$



$E \rightarrow ( E ) | E + T | T * F | a$

$F \rightarrow ( E ) | a$

$T \rightarrow ( E ) | T * F | a$

In [9]:
def remove_chain_rules(grammar: Grammar) -> Grammar:
    # Шаг 1: Построение N_A для каждого нетерминала A
    N = {nt: set([nt]) for nt in grammar.nonterminal}  # Начальное множество N_A={A}
    print("Начальные множества N_A:")
    for A in N:
        print(f"N_{A}: {N[A]}")
    
    changes = True
    while changes:  # Повторяем до тех пор, пока есть изменения
        changes = False
        for A in grammar.nonterminal:
            current_N = N[A].copy()  # Tекущее множество N_i-1
            for B in current_N:
                for rule in grammar.rules:
                    # Если правило имеет вид B -> C и C - нетерминал, то добавляем C в N_A
                    if rule.lhs == B and len(rule.rhs) == 1 and rule.rhs[0] in grammar.nonterminal:
                        if rule.rhs[0] not in N[A]:
                            N[A].add(rule.rhs[0])
                            changes = True
                            print(f"Добавляем {rule.rhs[0]} в N_{A} из правила {B} -> {rule.rhs[0]}")
    
    print("Итоговые множества N_A после всех добавлений:")
    for A in N:
        print(f"N_{A}: {N[A]}")

    # Шаг 2: Построение нового набора правил P'
    new_rules = []
    print("Новый набор правил P':")
    for A in grammar.nonterminal:
        for rule in grammar.rules:
            # Добавляем правило A -> α, если оно не является цепным правилом
            if len(rule.rhs) != 1 or rule.rhs[0] not in grammar.nonterminal:
                # Если B входит в N_A, добавляем A -> α для каждого такого B
                if rule.lhs in N[A]:
                    new_rules.append(GrammarRule(A, rule.rhs.copy()))
                    print(f"Добавляем правило {A} -> {' '.join(rule.rhs)} потому что {rule.lhs} ∈ N_{A}")

    # Шаг 3: Создаем новую грамматику без цепных правил
    return Grammar(grammar.nonterminal, grammar.terminal, new_rules, grammar.axiom)

# Теперь можно использовать функцию для создания новой грамматики без цепных правил
new_grammar = remove_chain_rules(grammar)
new_grammar.show()

Начальные множества N_A:
N_E: {'E'}
N_T: {'T'}
N_F: {'F'}
Добавляем T в N_E из правила E -> T
Добавляем F в N_T из правила T -> F
Добавляем F в N_E из правила T -> F
Итоговые множества N_A после всех добавлений:
N_E: {'T', 'F', 'E'}
N_T: {'T', 'F'}
N_F: {'F'}
Новый набор правил P':
Добавляем правило E -> E + T потому что E ∈ N_E
Добавляем правило E -> T * F потому что T ∈ N_E
Добавляем правило E -> a потому что F ∈ N_E
Добавляем правило E -> ( E ) потому что F ∈ N_E
Добавляем правило T -> T * F потому что T ∈ N_T
Добавляем правило T -> a потому что F ∈ N_T
Добавляем правило T -> ( E ) потому что F ∈ N_T
Добавляем правило F -> a потому что F ∈ N_F
Добавляем правило F -> ( E ) потому что F ∈ N_F


$[E,F,T] [(,),+,-,a] E$



$E \rightarrow ( E ) | E + T | T * F | a$

$F \rightarrow ( E ) | a$

$T \rightarrow ( E ) | T * F | a$